# Singapore University Acceptance Classification Model

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

## Load Data

In [2]:
df = pd.read_csv("data/2023-NUS.csv", header=1)
df.head(30)

,Nationality,Education Track,Results (GPA),A-level Results,IB Score,University,Course Name,Results
0,Singaporean,Poly,3.94,NaN,NaN,NUS,Accountancy,Accepted by direct entry/discovery day
1,Singaporean,Poly,3.00,NaN,NaN,NUS,Applied Ai And Analytics,Rejected
2,Singaporean,Poly,3.69,NaN,NaN,NUS,Architecture,Accepted by direct entry/discovery day
3,Singaporean,JC,NaN,76.25,NaN,NUS,Architecture,Accepted by ABA
4,Singaporean,JC,NaN,78.75,NaN,NUS,Bba,Rejected
5,Singaporean,Poly,3.87,NaN,NaN,NUS,Bba,Accepted by direct entry/discovery day
6,Singaporean,Poly,3.92,NaN,NaN,NUS,Business Administration,Accepted by direct entry/discovery day
7,Singaporean,Poly,3.89,NaN,NaN,NUS,Business Administration,Accepted by direct entry/discovery day
8,Singaporean,Poly,3.87,NaN,NaN,NUS,Business Administration,Accepted by direct entry/discovery day
9,Singaporean,Poly,3.88,NaN,NaN,NUS,Business Administration,Accepted by direct entry/discovery day


In [4]:
np.unique(df["Nationality"].to_numpy())
np.unique(df["Education Track"].to_numpy())

array(['IB', 'JC', 'Poly'], dtype=object)

## Data Preprocessing

### Data Overview

- **Features**
  - Nationality
  - Education
  - Results (GPA / A-Level / IB)
  - Course
- **Target**: Results

### Preprocessing Steps

1. One-hot-encoding nationality.
2. One-hot-encoding education.
3. Grades???
4. Categorize course?
5. Convert results to 0 / 1 using Regex.